# 🪪 ID Card OCR — Arabic & French
**Using PaddleOCR with multi-language support (Arabic + French)**

This notebook walks through:
1. Installing dependencies
2. Pre-processing the ID card image
3. Running OCR in Arabic and French separately
4. Merging and parsing results into structured fields
5. Visualising bounding boxes on the card
6. Exporting results to JSON

> ✅ Tested on Moroccan, Algerian, and Tunisian national ID cards.

## 📦 Cell 1 — Install Dependencies

In [ ]:
# Run once — restart kernel after installation
!pip install paddlepaddle paddleocr opencv-python-headless pillow arabic-reshaper python-bidi matplotlib -q

## 📚 Cell 2 — Imports

In [ ]:
import cv2
import re
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from pathlib import Path
from PIL import Image, ImageDraw, ImageFont
from paddleocr import PaddleOCR

# Arabic text rendering helpers
import arabic_reshaper
from bidi.algorithm import get_display

print('✅ All imports successful')

## ⚙️ Cell 3 — Configuration

In [ ]:
# ─── USER CONFIG — edit these ────────────────────────────────────────────────

IMAGE_PATH = "id_card.jpg"          # ← Path to your ID card image
CONFIDENCE_THRESHOLD = 0.55         # Drop detections below this score
USE_GPU = False                     # Set True if CUDA GPU is available
SAVE_ANNOTATED = True               # Save annotated output image
OUTPUT_JSON = "id_card_result.json" # Output JSON filename

# ─── Field keywords in BOTH languages ────────────────────────────────────────
FIELD_KEYWORDS = {
    "last_name": [
        # French
        "nom", "nom de famille", "surname",
        # Arabic
        "الاسم العائلي", "اسم العائلة", "النسب"
    ],
    "first_name": [
        "prénom", "prenom", "given name",
        "الاسم الشخصي", "الاسم"
    ],
    "dob": [
        "date de naissance", "né le", "née le", "naissance",
        "تاريخ الازدياد", "تاريخ الميلاد"
    ],
    "pob": [
        "lieu de naissance", "né à", "née à",
        "مكان الازدياد", "مكان الميلاد"
    ],
    "expiry": [
        "valable jusqu'au", "expire le", "date d'expiration", "valid until",
        "صالحة إلى", "تاريخ الانتهاء"
    ],
    "id_number": [
        "n° carte", "numéro", "n°", "cin", "id no",
        "رقم البطاقة", "رقم التعريف الوطني"
    ],
    "address": [
        "adresse", "domicile", "résidence",
        "العنوان", "محل السكنى"
    ],
    "nationality": [
        "nationalité", "nationalite",
        "الجنسية"
    ],
    "sex": [
        "sexe", "genre",
        "الجنس"
    ],
}

# Date regex (handles DD/MM/YYYY, YYYY-MM-DD, DD-MM-YYYY, etc.)
DATE_PATTERN = r"\b(\d{1,2}[\/-]\d{1,2}[\/-]\d{2,4}|\d{4}[\/-]\d{2}[\/-]\d{2})\b"

# ID number pattern — adjust to your country format
# Morocco CIN: 1-2 letters + 5-6 digits  |  Algeria NNI: 18 digits
ID_PATTERN = r"\b([A-Z]{1,2}\d{5,6}|\d{9,18})\b"

print("✅ Configuration loaded")

## 🖼️ Cell 4 — Load & Preview Raw Image

In [ ]:
def load_image(path: str) -> np.ndarray:
    img = cv2.imread(path)
    if img is None:
        raise FileNotFoundError(f"Cannot load image: {path}")
    return img

def show_image(img: np.ndarray, title: str = "", figsize=(14, 8)):
    rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=figsize)
    plt.imshow(rgb)
    plt.title(title, fontsize=14)
    plt.axis("off")
    plt.tight_layout()
    plt.show()

raw_img = load_image(IMAGE_PATH)
show_image(raw_img, "🖼️ Original ID Card")
print(f"Image shape: {raw_img.shape}  (H × W × C)")

## 🔧 Cell 5 — Pre-processing Pipeline

In [ ]:
def resize_if_small(img: np.ndarray, min_side: int = 900) -> np.ndarray:
    """Upscale so the shortest side is at least `min_side` pixels."""
    h, w = img.shape[:2]
    scale = max(min_side / min(h, w), 1.0)
    if scale > 1.0:
        img = cv2.resize(img, (int(w * scale), int(h * scale)),
                         interpolation=cv2.INTER_CUBIC)
    return img


def denoise(img: np.ndarray) -> np.ndarray:
    return cv2.fastNlMeansDenoisingColored(img, None, 10, 10, 7, 21)


def sharpen(img: np.ndarray) -> np.ndarray:
    kernel = np.array([[0, -1,  0],
                       [-1,  5, -1],
                       [0, -1,  0]])
    return cv2.filter2D(img, -1, kernel)


def deskew(img: np.ndarray) -> np.ndarray:
    """Correct slight rotation using dominant edge angle."""
    gray  = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180,
                             threshold=80, minLineLength=80, maxLineGap=10)
    if lines is None:
        return img
    angles = []
    for line in lines:
        x1, y1, x2, y2 = line[0]
        angle = np.degrees(np.arctan2(y2 - y1, x2 - x1))
        if abs(angle) < 45:
            angles.append(angle)
    if not angles:
        return img
    median_angle = np.median(angles)
    if abs(median_angle) < 0.5:
        return img
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((w / 2, h / 2), median_angle, 1.0)
    return cv2.warpAffine(img, M, (w, h),
                          flags=cv2.INTER_CUBIC,
                          borderMode=cv2.BORDER_REPLICATE)


def enhance_contrast(img: np.ndarray) -> np.ndarray:
    """CLAHE contrast enhancement on the L channel."""
    lab  = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l     = clahe.apply(l)
    return cv2.cvtColor(cv2.merge([l, a, b]), cv2.COLOR_LAB2BGR)


def preprocess(img: np.ndarray) -> np.ndarray:
    img = resize_if_small(img)
    img = denoise(img)
    img = deskew(img)
    img = enhance_contrast(img)
    img = sharpen(img)
    return img


processed_img = preprocess(raw_img)
show_image(processed_img, "✅ Pre-processed Image")
print(f"Processed shape: {processed_img.shape}")

## 🔤 Cell 6 — Initialise OCR Engines

PaddleOCR does not support Arabic + French in a single model pass.  
We run **two separate engines** and merge their results:
- `arabic` model — detects Arabic script
- `french` model (latin script via `fr`) — detects French / Latin text

In [ ]:
print("⏳ Loading Arabic OCR engine...")
ocr_ar = PaddleOCR(
    use_angle_cls=True,
    lang="ar",          # Arabic model
    use_gpu=USE_GPU,
    show_log=False,
)
print("✅ Arabic OCR engine ready")

print("⏳ Loading French / Latin OCR engine...")
ocr_fr = PaddleOCR(
    use_angle_cls=True,
    lang="fr",          # French (Latin) model
    use_gpu=USE_GPU,
    show_log=False,
)
print("✅ French OCR engine ready")

## 🔍 Cell 7 — Run Both OCR Engines

In [ ]:
def extract_lines(ocr_engine: PaddleOCR,
                  img: np.ndarray,
                  language: str,
                  threshold: float = CONFIDENCE_THRESHOLD) -> list:
    """
    Run OCR and return filtered list of dicts:
      {text, confidence, bbox, language, bbox_center}
    """
    results = ocr_engine.ocr(img, cls=True)
    lines   = []
    for page in results:
        if not page:
            continue
        for item in page:
            bbox, (text, conf) = item
            text = text.strip()
            if not text or conf < threshold:
                continue
            pts = np.array(bbox)
            cx  = float(pts[:, 0].mean())
            cy  = float(pts[:, 1].mean())
            lines.append({
                "text":       text,
                "confidence": round(float(conf), 4),
                "bbox":       bbox,
                "language":   language,
                "bbox_center": (cx, cy),
            })
    return lines


def is_arabic(text: str) -> bool:
    """Return True if the string contains Arabic characters."""
    return bool(re.search(r'[\u0600-\u06FF\u0750-\u077F]', text))


def deduplicate(ar_lines: list, fr_lines: list,
                iou_threshold: float = 0.4) -> list:
    """
    Merge Arabic and French OCR outputs.
    If two detections overlap substantially (same bounding box region),
    prefer the one whose language matches the script of the recognised text.
    """
    def bbox_to_rect(bbox):
        pts = np.array(bbox)
        x1, y1 = pts.min(axis=0)
        x2, y2 = pts.max(axis=0)
        return x1, y1, x2, y2

    def iou(b1, b2):
        ax1, ay1, ax2, ay2 = bbox_to_rect(b1)
        bx1, by1, bx2, by2 = bbox_to_rect(b2)
        ix1 = max(ax1, bx1); iy1 = max(ay1, by1)
        ix2 = min(ax2, bx2); iy2 = min(ay2, by2)
        inter = max(0, ix2 - ix1) * max(0, iy2 - iy1)
        union = (ax2-ax1)*(ay2-ay1) + (bx2-bx1)*(by2-by1) - inter
        return inter / union if union > 0 else 0.0

    merged = list(ar_lines)          # start with Arabic results
    for fr_item in fr_lines:
        overlap = False
        for ar_item in ar_lines:
            if iou(fr_item["bbox"], ar_item["bbox"]) > iou_threshold:
                overlap = True
                # Keep the result whose model matches the detected script
                if is_arabic(ar_item["text"]) and not is_arabic(fr_item["text"]):
                    pass   # keep Arabic result already in merged
                elif not is_arabic(ar_item["text"]) and not is_arabic(fr_item["text"]):
                    # Both Latin — keep higher confidence
                    if fr_item["confidence"] > ar_item["confidence"]:
                        merged = [fr_item if x is ar_item else x for x in merged]
                break
        if not overlap:
            merged.append(fr_item)

    # Sort top-to-bottom, then left-to-right (reading order)
    merged.sort(key=lambda x: (round(x["bbox_center"][1] / 30), x["bbox_center"][0]))
    return merged


print("⏳ Running Arabic OCR...")
ar_lines = extract_lines(ocr_ar, processed_img, language="ar")
print(f"   Found {len(ar_lines)} Arabic detections")

print("⏳ Running French OCR...")
fr_lines = extract_lines(ocr_fr, processed_img, language="fr")
print(f"   Found {len(fr_lines)} French detections")

print("⏳ Merging & deduplicating...")
all_lines = deduplicate(ar_lines, fr_lines)
print(f"✅ Total unique detections after merge: {len(all_lines)}")

## 📋 Cell 8 — Display Raw OCR Output

In [ ]:
def render_arabic(text: str) -> str:
    """Reshape + apply BiDi so Arabic renders correctly in matplotlib."""
    if is_arabic(text):
        return get_display(arabic_reshaper.reshape(text))
    return text


print(f"{'#':>3}  {'Lang':>4}  {'Conf':>6}  Text")
print("-" * 70)
for i, item in enumerate(all_lines, 1):
    display_text = render_arabic(item['text'])
    print(f"{i:>3}  {item['language']:>4}  {item['confidence']:>6.2f}  {display_text}")

## 🗂️ Cell 9 — Field Parsing

In [ ]:
def parse_id_fields(lines: list) -> dict:
    """
    Extract structured fields from the merged OCR lines using:
    - Keyword matching (bilingual)
    - Regex for dates, ID numbers, MRZ
    """
    fields = {k: None for k in FIELD_KEYWORDS}
    fields["mrz"]       = []
    fields["all_dates"] = []
    fields["all_text"]  = [item["text"] for item in lines]

    MRZ_RE = re.compile(r'^[A-Z0-9<]{30,}$')

    for i, item in enumerate(lines):
        text  = item["text"]
        lower = text.lower()

        # ── MRZ ───────────────────────────────────────────────────
        if MRZ_RE.match(text):
            fields["mrz"].append(text)
            continue

        # ── Dates ─────────────────────────────────────────────────
        for m in re.finditer(DATE_PATTERN, text):
            fields["all_dates"].append(m.group())

        # ── ID number ─────────────────────────────────────────────
        if not fields["id_number"]:
            m = re.search(ID_PATTERN, text)
            if m:
                fields["id_number"] = m.group(1)

        # ── Keyword fields ────────────────────────────────────────
        for field, keywords in FIELD_KEYWORDS.items():
            if fields[field]:             # already found
                continue
            if any(kw in lower or kw in text for kw in keywords):
                # Value is either after ':' on same line or on next line
                if ":" in text:
                    value = text.split(":", 1)[-1].strip()
                elif "،" in text:         # Arabic comma
                    value = text.split("،", 1)[-1].strip()
                elif i + 1 < len(lines):
                    value = lines[i + 1]["text"]
                else:
                    value = text
                if value:
                    fields[field] = value

    # Assign dates heuristically if not found via keywords
    dates = fields["all_dates"]
    if dates:
        if not fields["dob"]:
            fields["dob"]    = dates[0]
        if not fields["expiry"] and len(dates) >= 2:
            fields["expiry"] = dates[-1]

    return fields


parsed = parse_id_fields(all_lines)

print("\n" + "═" * 50)
print("          🪪  PARSED ID CARD FIELDS")
print("═" * 50)
for field, value in parsed.items():
    if field in ("all_text", "mrz", "all_dates"):
        continue
    display_val = render_arabic(str(value)) if value else "—"
    print(f"  {field:<15}: {display_val}")

if parsed["all_dates"]:
    print(f"\n  All dates found : {parsed['all_dates']}")

if parsed["mrz"]:
    print("\n  MRZ Lines:")
    for line in parsed["mrz"]:
        print(f"    {line}")

## 🎨 Cell 10 — Visualise OCR Bounding Boxes

In [ ]:
COLOUR_MAP = {"ar": (255, 80,  80),   # red for Arabic
              "fr": (50,  180, 255)}  # blue for French

def annotate_image_pil(img_bgr: np.ndarray, lines: list) -> np.ndarray:
    """
    Draw coloured bounding boxes + text labels using PIL so Arabic
    characters render correctly.
    """
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    pil_img = Image.fromarray(img_rgb)
    draw    = ImageDraw.Draw(pil_img)

    # Try to load a font that supports Arabic; fall back to default
    try:
        font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 18)
    except Exception:
        font = ImageFont.load_default()

    for item in lines:
        pts    = [tuple(map(int, pt)) for pt in item["bbox"]]
        colour = COLOUR_MAP.get(item["language"], (0, 220, 0))

        # Draw polygon
        draw.polygon(pts, outline=colour)
        for j in range(4):
            draw.line([pts[j], pts[(j+1) % 4]], fill=colour, width=2)

        # Label text (reshaped if Arabic)
        label = render_arabic(item["text"])
        x, y  = pts[0]
        draw.text((x, max(y - 20, 0)), label[:40], fill=colour, font=font)

    return cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)


annotated = annotate_image_pil(processed_img, all_lines)

# Legend
ar_patch = mpatches.Patch(color=(1, 0.31, 0.31), label='Arabic detections')
fr_patch = mpatches.Patch(color=(0.20, 0.71, 1.0), label='French detections')

plt.figure(figsize=(16, 10))
plt.imshow(cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB))
plt.legend(handles=[ar_patch, fr_patch], loc='lower right', fontsize=12)
plt.title("OCR Detections — Arabic (red) & French (blue)", fontsize=14)
plt.axis("off")
plt.tight_layout()
plt.show()

if SAVE_ANNOTATED:
    out_path = Path(IMAGE_PATH).stem + "_annotated.jpg"
    cv2.imwrite(out_path, annotated)
    print(f"💾 Annotated image saved → {out_path}")

## 📊 Cell 11 — Confidence Distribution Chart

In [ ]:
ar_conf = [l["confidence"] for l in all_lines if l["language"] == "ar"]
fr_conf = [l["confidence"] for l in all_lines if l["language"] == "fr"]

fig, axes = plt.subplots(1, 2, figsize=(14, 4))

for ax, confs, label, colour in [
    (axes[0], ar_conf, "Arabic",  "#FF5050"),
    (axes[1], fr_conf, "French",  "#32B4FF"),
]:
    ax.hist(confs, bins=20, range=(0, 1), color=colour, edgecolor="white", alpha=0.85)
    ax.axvline(np.mean(confs) if confs else 0, color="black", linestyle="--",
               label=f"Mean: {np.mean(confs):.2f}" if confs else "No data")
    ax.set_title(f"{label} Confidence Distribution", fontsize=13)
    ax.set_xlabel("Confidence"); ax.set_ylabel("Count")
    ax.legend()

plt.tight_layout()
plt.show()

## 💾 Cell 12 — Export Results to JSON

In [ ]:
def serialise(obj):
    """Make bbox (list of lists of floats) JSON-safe."""
    if isinstance(obj, np.floating):
        return float(obj)
    if isinstance(obj, np.integer):
        return int(obj)
    raise TypeError(f"Not serialisable: {type(obj)}")


output = {
    "source_image":  IMAGE_PATH,
    "parsed_fields": {
        k: v for k, v in parsed.items()
        if k not in ("all_text",)
    },
    "raw_lines": [
        {
            **line,
            "bbox": [[float(x), float(y)] for x, y in line["bbox"]],
            "bbox_center": [float(line["bbox_center"][0]),
                            float(line["bbox_center"][1])],
        }
        for line in all_lines
    ],
}

with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
    json.dump(output, f, ensure_ascii=False, indent=2, default=serialise)

print(f"✅ Results saved → {OUTPUT_JSON}")
print(f"   Total lines exported : {len(all_lines)}")
print(f"   Fields extracted     : {sum(1 for v in parsed.values() if v and v != [] and isinstance(v, str))}")

## 🔁 Cell 13 — Batch Processing (Multiple Cards)

In [ ]:
def process_batch(image_paths: list[str]) -> list[dict]:
    """
    Process multiple ID card images and return a list of results.
    Each result contains the source path and parsed fields.
    """
    results = []
    for path in image_paths:
        print(f"\n--- Processing: {path} ---")
        try:
            img   = load_image(path)
            proc  = preprocess(img)
            ar_l  = extract_lines(ocr_ar, proc, "ar")
            fr_l  = extract_lines(ocr_fr, proc, "fr")
            lines = deduplicate(ar_l, fr_l)
            flds  = parse_id_fields(lines)
            results.append({"image": path, "fields": flds, "raw_lines": lines})
            print(f"  ✅ id_number={flds.get('id_number')}  "
                  f"dob={flds.get('dob')}  expiry={flds.get('expiry')}")
        except Exception as e:
            print(f"  ❌ Error: {e}")
            results.append({"image": path, "error": str(e)})
    return results


# ── Example usage (uncomment and edit paths) ──────────────────────────────────
# batch_images = ["card1.jpg", "card2.jpg", "card3.png"]
# batch_results = process_batch(batch_images)
#
# with open("batch_results.json", "w", encoding="utf-8") as f:
#     json.dump(batch_results, f, ensure_ascii=False, indent=2)
# print("Batch complete — results saved to batch_results.json")

print("Batch processing function defined. Uncomment the example lines above to run it.")

---
## 📝 Notes & Tips

| Topic | Guidance |
|---|---|
| **Languages** | `lang="ar"` for Arabic, `lang="fr"` for French/Latin. PaddleOCR 2.x does not support mixed-script in a single engine. |
| **GPU** | Set `USE_GPU = True` and install `paddlepaddle-gpu` for 5–10× speedup. |
| **ID number regex** | `ID_PATTERN` in Cell 3 — update to match your country's format (Morocco CIN, Algeria NNI, Tunisia CIN, etc.). |
| **Field keywords** | Extend `FIELD_KEYWORDS` in Cell 3 if your card uses different label wording. |
| **MRZ parsing** | For full ICAO 9303 MRZ decoding, add the `mrz` library: `pip install mrz`. |
| **Poor results?** | Try increasing image resolution before scanning, or crop the card tightly before passing to `preprocess()`. |